In [1]:
# Import the neccesary libraries needed
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
from numpy import insert
import os
import time

# keras tuner for hyperparameter tuning
import keras_tuner as kt

# tensorflow Libraries
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import sparse_categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [2]:
# Noise Reduction Dataset
train = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port_FinalCleaned_RollReady.csv')
train2 = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002_port_FinalCleaned_RollReady.csv')
test = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd_FinalCleaned_RollReady.csv')
test2 = pd.read_csv('D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002_stbd_FinalCleaned_RollReady.csv')

In [3]:
# Split up the features and labels from both the training and testing datasets
#x_train=train.iloc[:,4:24]

#x_train=train.iloc[:,3:24] # sample time delay through Q10

x_train1=train.iloc[:,3:25] # sample time delay through roll
x_train2 = train2.iloc[:, 3:25]
#x_train=train38.iloc[:,3:24]

x_train1['Amplitude']=train.iloc[:,29]
x_train2['Amplitude']=train2.iloc[:,29]
#x_train['Amplitude']=train38.iloc[:,29]


#x_test=test.iloc[:,4:24]
#x_test=test.iloc[:,3:24]    # sample time delay through Q10

x_test=test.iloc[:,3:25]    # sample time delay through roll
x_test2=test2.iloc[:,3:25]    # sample time delay through roll
#x_test['Roll'] = x_test['Roll']


x_test['Amplitude']=test.iloc[:,29]
x_test2['Amplitude']=test2.iloc[:,29]
#x_test['Amplitude']=test38.iloc[:,29]

y_train1=train.iloc[:,26]
y_train2=train2.iloc[:,26]
#y_train=train38.iloc[:,26]

y_test=test.iloc[:,26]
y_test2=test2.iloc[:,26]
#y_test=test38.iloc[:,26]

In [4]:
xframes = [x_train1, x_train2]
x_train = pd.concat(xframes, sort=False)
yframes = [y_train1, y_train2]
y_train = pd.concat(yframes, sort=False)

In [5]:
# Scale the Features and Labels from [-1,1]

s1=MinMaxScaler(feature_range=(0,1))
#s1=MinMaxScaler(feature_range=(0,1))
x_train_scale=s1.fit_transform(x_train)

#s2=MinMaxScaler(feature_range=(0,1))
#s2=MinMaxScaler(feature_range=(0,1))
x_test_scale=s1.transform(x_test)
x_test_scale=x_test_scale
x_test_scale[:, 21] = -x_test_scale[:, 21]
x_test2_scale=s1.transform(x_test2)
x_test2_scale[:, 21] = -x_test2_scale[:, 21]

# Scale DOA's from (0 to 1)
#s3=MinMaxScaler(feature_range=(0,1))
#y_train_scale = s3.fit_transform(train[['DOA']])
y_train_scale= y_train.to_numpy()

#s4=MinMaxScaler(feature_range=(0,1))
#y_test_scale = s4.fit_transform(test[['DOA']])
y_test_scale = y_test.to_numpy()
y_test2_scale = y_test2.to_numpy()
print(len(x_train_scale))
print(len(y_train_scale))

7622237
7622237


In [6]:
# Shift the DOA to the left by one to use the current DOA label with the current I and Q data.
# Normally the TimeseriesGenerator function uses past values to predict the future but we would like current data to help predict the current samples' DOA
print(len(y_train_scale))
print(len(y_test_scale))
y_train_scale=insert(y_train_scale, 0, 0)
y_train_scale=np.delete(y_train_scale, -1)
y_test_scale=insert(y_test_scale, 0,0)
y_test_scale=np.delete(y_test_scale, -1)
y_test2_scale=insert(y_test2_scale, 0,0)
y_test2_scale=np.delete(y_test2_scale, -1)
print(y_train_scale)
print(len(y_train_scale))
print(y_test_scale)
print(len(y_test_scale))

7622237
3382112
[ 0.00000000e+00 -3.05400003e+00 -5.40000005e-02 ...  8.02140007e+01
  8.02440007e+01  8.02680007e+01]
7622237
[ 0.          3.69000003  2.39400002 ... 76.59600067 76.35000066
 76.32600066]
3382112


In [8]:
# Create the windows neccesary for the LSTM model within tensorflow keras
# Below are the arguments of the timeseries_dataset_from_array function
# The function takes a numpy array and makes a timeseries out of it
NumSampsPerPing= 4301-24+1
data=x_train_scale # the data to make the windows
targets=y_train_scale # time steps in the data (don't need)
sequence_length=5 # window length
sequence_stride=1# period between successive output sequences
sampling_rate=1 # period between successive individual timesteps     within sequences
batch_size=32 # number of time series samples in each batch
shuffle=False #shuffle the data before making the windows
seed=None # is related to shuffle
start_index=None # is related to shuffle
end_index=None # is related to shuffle
n_features=23

test_data=x_test_scale
test_targets=y_test_scale
test2_data = x_test2_scale
test2_targets = y_test2_scale
dataRows = data.shape[0]
dataRows

7622237

In [9]:
#validationStartRow = dataRows - NumSampsPerPing*281
validationStartRow = dataRows - 200*1000
#validationStartRow = dataRows - 8*1000

print(validationStartRow, ',',dataRows)

train_data = data[:validationStartRow-1]
#train_data = data
#   train_data.shape[0]
train_targets = targets[:validationStartRow-1]
#train_targets = targets
#   train_targets.shape[0]
val_data = data[validationStartRow:]
#   val_data.shape[0], dataRows-validationStartRow
y_val_scale = y_train_scale[validationStartRow:]

val_targets = targets[validationStartRow:]

7422237 , 7622237


In [10]:


#inputs=TimeseriesGenerator(data, targets,sequence_length, batch_size)
train_inputs=TimeseriesGenerator(data=train_data,targets=train_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=batch_size, shuffle=True)
trainentire_inputs = TimeseriesGenerator(data=data,targets=targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256)

val_inputs = TimeseriesGenerator(data=val_data,targets=val_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256, shuffle=False)

#test_inputs=TimeseriesGenerator(test_data[0:40000], -1*test_targets[0:40000],sequence_length, batch_size)
test_inputs=TimeseriesGenerator(data=test_data,targets=test_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256)
test_inputs2=TimeseriesGenerator(data=test2_data,targets=test2_targets,length=sequence_length,sampling_rate=sampling_rate,stride=sequence_stride,batch_size=256)

#inputs=tf.keras.preprocessing.timeseries_dataset_from_array(data, targets, sequence_length, sequence_stride, sampling_rate, batch_size, shuffle)
#print(type(inputs))

#print('done')

In [11]:
#model = keras.models.load_model('model/')
#model = keras.models.load_model('model_3_13_22_epoch4/model/')
model = keras.models.load_model(filepath="D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\SavedModels\\MB\\Mar10_2022\\model_row31.h5")

OSError: No file or directory found at D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\SavedModels\MB\Mar10_2022\model_row31.h5

In [ ]:
#trainpredicted=model.predict(train_inputs, verbose=1)
#valpredicted=model.predict(val_inputs, verbose=1)
#trainentirepredicted=model.predict(trainentire_inputs, verbose=1)
predicted038=model.predict(test_inputs, verbose=1)
predicted002=model.predict(test_inputs2, verbose=1)
#trainpredicted.shape, valpredicted.shape, predicted.shape, trainentirepredicted.shape